<h1>导入数据</h1>

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import time
from datetime import datetime
from tqdm import tqdm
from time import strftime, localtime
from scipy import stats
from scipy.spatial import distance
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier
from joblib import dump, load
from tensorflow.keras.backend import epsilon
from itertools import combinations
from imblearn.over_sampling import SMOTE, ADASYN
from tsfresh import extract_relevant_features, extract_features
from tsfresh.feature_extraction import EfficientFCParameters, MinimalFCParameters

<h1>数据处理</h1>

<h2>数据预处理</h2>

<h3>conf_info数据预处理</h3>

In [2]:
cons_info = pd.read_csv("../data/训练组_比特币挖矿_档案明细（20211220）.csv", index_col=0, quotechar='"', encoding="gbk")
cons_info

,ID,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CANCEL_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,TMP_DATE,IS_FLAG
,,,,,,,,,,,,,,
1,329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
2,329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
3,329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
4,329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
5,329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,NaN,240.0,2005/6/18,非临时用电,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8836,2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/2 16:12:00,NaN,120.0,2019/12/2 16:12:00,非临时用电,NaN,0
8837,2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019/12/3 16:33:00,NaN,120.0,2019/12/3 16:33:00,非临时用电,NaN,0
8838,2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/18 14:42:00,NaN,120.0,2019/12/18 14:42:00,非临时用电,NaN,0


In [3]:
cons_info = cons_info.set_index("ID").drop(columns=["CANCEL_DATE", "TMP_DATE"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005/6/18,240.0,2005/6/18,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/2 16:12:00,120.0,2019/12/2 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019/12/3 16:33:00,120.0,2019/12/3 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019/12/18 14:42:00,120.0,2019/12/18 14:42:00,非临时用电,0


In [4]:
cons_info["BUILD_DATE"] = pd.to_datetime(cons_info["BUILD_DATE"])
cons_info["LAST_CHK_DATE"] = pd.to_datetime(cons_info["LAST_CHK_DATE"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,NaN,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,NaN,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0


In [5]:
cons_info["SHIFT_NO"] = cons_info["SHIFT_NO"].fillna(0)
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0


In [6]:
cons_info["now"] = "2021-12-31"
cons_info["now"] = pd.to_datetime(cons_info["now"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG,now
ID,,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0,2021-12-31
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0,2021-12-31
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0,2021-12-31


In [7]:
cons_info["live_days"] = (cons_info["now"] - cons_info["BUILD_DATE"]).map(lambda x: x.days)
cons_info["check_days"] = (cons_info["now"] - cons_info["LAST_CHK_DATE"]).map(lambda x: x.days)
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,IS_FLAG,now,live_days,check_days
ID,,,,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,2005-06-18 00:00:00,240.0,2005-06-18 00:00:00,非临时用电,0,2021-12-31,6040,6040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-02 16:12:00,120.0,2019-12-02 16:12:00,非临时用电,0,2021-12-31,759,759
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,2019-12-03 16:33:00,120.0,2019-12-03 16:33:00,非临时用电,0,2021-12-31,758,758
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,2019-12-18 14:42:00,120.0,2019-12-18 14:42:00,非临时用电,0,2021-12-31,743,743


In [8]:
cons_info = cons_info.drop(columns=["BUILD_DATE", "LAST_CHK_DATE", "now"])
cons_info

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,TMP_NAME,IS_FLAG,live_days,check_days
ID,,,,,,,,,,,
329465205,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,240.0,非临时用电,0,6040,6040
329465245,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,4.0,4.0,1.0,240.0,非临时用电,0,6040,6040
329465357,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,240.0,非临时用电,0,6040,6040
329465360,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,2.0,2.0,1.0,240.0,非临时用电,0,6040,6040
329465439,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,8.0,8.0,1.0,240.0,非临时用电,0,6040,6040
...,...,...,...,...,...,...,...,...,...,...,...
2862711473,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,120.0,非临时用电,0,759,759
2864274650,乡村居民生活用电,交流220V,居民合表电价(不满1千伏）,13.0,13.0,0.0,120.0,非临时用电,0,758,758
2872313739,乡村居民生活用电,交流380V,居民合表电价(不满1千伏）,39.0,39.0,0.0,120.0,非临时用电,0,743,743


In [9]:
cons_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8840 entries, 329465205 to 2529558569
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ELEC_TYPE_NAME  8840 non-null   object 
 1   VOLT_NAME       8840 non-null   object 
 2   PRC_NAME        8840 non-null   object 
 3   CONTRACT_CAP    8840 non-null   float64
 4   RUN_CAP         8840 non-null   float64
 5   SHIFT_NO        8840 non-null   float64
 6   CHK_CYCLE       8838 non-null   float64
 7   TMP_NAME        8840 non-null   object 
 8   IS_FLAG         8840 non-null   int64  
 9   live_days       8840 non-null   int64  
 10  check_days      8840 non-null   int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 828.8+ KB


In [10]:
# 因为看过了这部分特征重要性为0，所以丢掉
cons_info = cons_info.drop(columns=["ELEC_TYPE_NAME", "VOLT_NAME", "PRC_NAME", "TMP_NAME"])
cons_info

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days
ID,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040
329465245,4.0,4.0,1.0,240.0,0,6040,6040
329465357,8.0,8.0,1.0,240.0,0,6040,6040
329465360,2.0,2.0,1.0,240.0,0,6040,6040
329465439,8.0,8.0,1.0,240.0,0,6040,6040
...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759
2864274650,13.0,13.0,0.0,120.0,0,758,758
2872313739,39.0,39.0,0.0,120.0,0,743,743


In [11]:
cons_info = pd.get_dummies(cons_info)
cons_info

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days
ID,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040
329465245,4.0,4.0,1.0,240.0,0,6040,6040
329465357,8.0,8.0,1.0,240.0,0,6040,6040
329465360,2.0,2.0,1.0,240.0,0,6040,6040
329465439,8.0,8.0,1.0,240.0,0,6040,6040
...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759
2864274650,13.0,13.0,0.0,120.0,0,758,758
2872313739,39.0,39.0,0.0,120.0,0,743,743


In [12]:
cons_info["CHK_CYCLE"] = cons_info["CHK_CYCLE"].fillna(cons_info["CHK_CYCLE"].median())
cons_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8840 entries, 329465205 to 2529558569
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CONTRACT_CAP  8840 non-null   float64
 1   RUN_CAP       8840 non-null   float64
 2   SHIFT_NO      8840 non-null   float64
 3   CHK_CYCLE     8840 non-null   float64
 4   IS_FLAG       8840 non-null   int64  
 5   live_days     8840 non-null   int64  
 6   check_days    8840 non-null   int64  
dtypes: float64(4), int64(3)
memory usage: 552.5 KB


<h3>elec_month数据预处理</h3>

<h4>定义一些特征工程</h4>

In [13]:
def entropy(data, bins: int = 10) -> float:
    """
    计算一个序列的熵\n
    """
    nums = np.histogram(data, bins=bins)[0]
    probs = nums / sum(nums)
    return stats.entropy(probs)

In [14]:
def entropy_df(df: pd.DataFrame, bins: int = 10) -> float:
    """
    计算一段时间子电量的熵\n
    """
    entropys = df.apply(lambda x: entropy(x), axis=1)
    return entropys.mean()

In [15]:
def std_df(df: pd.DataFrame) -> float:
    """
    计算一段时间子电量的标准差\n
    """
    return df.std(axis=1).mean()

In [16]:
def cosine(a, b) -> float:
    """
    计算余弦距离\n
    """
    if np.linalg.norm(a) != 0 and np.linalg.norm(b) != 0:
        return distance.cosine(a, b)
    elif np.linalg.norm(a) == 0 and np.linalg.norm(b) == 0:
        return 0
    else:
        return 1

In [17]:
def diff_median(df: pd.DataFrame) -> float:
    """
    计算一个df的两列数值差的中位数\n
    """
    results = []
    for i, j in combinations(list(range(df.shape[1])), r=2):
        result = (df.iloc[:, i] - df.iloc[:, j]).apply(lambda x: abs(x)).median()
        results.append(result)
    return sum(results)

In [18]:
def devide_median(df: pd.DataFrame) -> float:
    """
    计算两个列的相除的中位数\n
    """
    results = []
    for i, j in combinations(list(range(df.shape[1])), r=2):
        result = (df.iloc[:, 0] / (df.iloc[:, 1] + epsilon())).median()
        results.append(result)
    return sum(results)

In [19]:
def variation(df: pd.DataFrame) -> float:
    """
    计算变异系数\n
    """
    tt =  (epsilon() + df.std(axis=1)) / (epsilon() + df.mean(axis=1))
    tt[tt.apply(lambda x: np.isinf(x))] = 0
    return tt.mean(axis=0)

<h4>读取数据并预处理</h4>

In [20]:
cons_info["pq_z_mean"] = 0
cons_info["pq_z_median"] = 0
cons_info["pq_z_std"] = 0
cons_info["pq_z_std/mean"] = 0
cons_info["pq_z_entropy"] = 0
cons_info["pq_fg_distance"] = 0
cons_info["pq_gp_diatance"] = 0
cons_info["pq_fp_distance"] = 0
cons_info["pq_fg_cosine"] = 0
cons_info["pq_gp_cosine"] = 0
cons_info["pq_fp_cosine"] = 0
cons_info["pq_fz_cosine"] = 0
cons_info["pq_gz_cosine"] = 0
cons_info["pq_pz_cosine"] = 0
cons_info["variation_month"] = 0
cons_info["pq_z_mean_4500"] = 0
cons_info["variation_month_0085"] = 0

cons_info["pq_z_max"] = 0
cons_info["pq_z_min"] = 0

cons_info["entropy_month"] = 0
cons_info["std_month"] = 0

cons_info["diff_month"] = 0
cons_info["devide_month"] = 0

In [21]:
elec_month = pd.read_csv("../data/训练组_比特币挖矿_月用电明细（20211217）.csv")
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329769614,202001,615,161,0,776
1,329769614,202002,492,154,-1,645
2,329769614,202003,457,114,1,572
3,329769614,202004,522,171,0,693
4,329769614,202005,519,181,0,700
...,...,...,...,...,...,...
194475,2878334859,202106,107,67,-1,173
194476,2878334859,202107,170,112,1,283
194477,2878334859,202108,102,67,0,169
194478,2878334859,202109,112,65,-1,176


In [22]:
elec_month["ym"] = elec_month["ym"].astype(str)
elec_month = elec_month.groupby('id').apply(lambda x: x.sort_values('ym', ascending=True).fillna(method="ffill").fillna(method="bfill").fillna(0)).set_index(elec_month.index)
elec_month

,id,ym,pq_f,pq_g,pq_p,pq_z
0,329449398,202001,234,129,-1,362
1,329449398,202002,257,145,0,402
2,329449398,202003,167,116,1,284
3,329449398,202004,170,112,-1,281
4,329449398,202005,239,162,0,401
...,...,...,...,...,...,...
194475,2878334859,202106,107,67,-1,173
194476,2878334859,202107,170,112,1,283
194477,2878334859,202108,102,67,0,169
194478,2878334859,202109,112,65,-1,176


<h4>手工特征工程</h4>

In [23]:
for cons_id in tqdm(cons_info.index):
    df = elec_month[elec_month["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("ym").set_index("ym")
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    
    cons_info.loc[cons_id, "pq_z_mean"] = df["pq_z"].mean()
    cons_info.loc[cons_id, "pq_z_median"] = df["pq_z"].median()
    cons_info.loc[cons_id, "pq_z_std"] = df["pq_z"].std()
    cons_info.loc[(cons_info.index == cons_id) & (cons_info["pq_z_mean"] > 0), "pq_z_std/mean"] = cons_info["pq_z_std"] / cons_info["pq_z_mean"]
    cons_info.loc[cons_id, "pq_z_entropy"] = entropy(df["pq_z"])
    
    cons_info.loc[cons_id, "pq_fg_distance"] = distance.euclidean(df["pq_f"], df["pq_g"])
    cons_info.loc[cons_id, "pq_gp_diatance"] = distance.euclidean(df["pq_g"], df["pq_p"])
    cons_info.loc[cons_id, "pq_fp_distance"] = distance.euclidean(df["pq_f"], df["pq_p"])
    
    cons_info.loc[cons_id, "pq_fg_cosine"] = cosine(df["pq_f"], df["pq_g"])
    cons_info.loc[cons_id, "pq_gp_cosine"] = cosine(df["pq_g"], df["pq_p"])
    cons_info.loc[cons_id, "pq_fp_cosine"] = cosine(df["pq_f"], df["pq_p"])
    cons_info.loc[cons_id, "pq_fz_cosine"] = cosine(df["pq_f"], df["pq_z"])
    cons_info.loc[cons_id, "pq_gz_cosine"] = cosine(df["pq_g"], df["pq_z"])
    cons_info.loc[cons_id, "pq_pz_cosine"] = cosine(df["pq_p"], df["pq_z"])
    
    cons_info.loc[cons_id, "variation_month"] = variation(df[["pq_f", "pq_g", "pq_p"]])
    cons_info.loc[cons_id, "pq_z_mean_4500"] = (cons_info.loc[cons_id, "pq_z_mean"] > 4500).astype(float)
    cons_info.loc[cons_id, "variation_month_0085"] = (cons_info.loc[cons_id, "variation_month"] < 0.085).astype(float)
    
    cons_info.loc[cons_id, "pq_z_max"] = df["pq_z"].max()
    cons_info.loc[cons_id, "pq_z_min"] = df["pq_z"].min()
    
    cons_info.loc[cons_id, "entropy_month"] = entropy_df(df[["pq_f", "pq_g", "pq_p"]])
    cons_info.loc[cons_id, "std_month"] = std_df(df[["pq_f", "pq_g", "pq_p"]])
    
    cons_info.loc[cons_id, "diff_month"] = diff_median(df[["pq_f", "pq_g", "pq_p"]])
    cons_info.loc[cons_id, "devide_month"] = devide_median(df[["pq_f", "pq_g", "pq_p"]])    

    
cons_info

100%|████████████████████████████████████████████████████████████████████████████████| 8840/8840 [02:47<00:00, 52.68it/s]


,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,pq_z_mean,pq_z_median,pq_z_std,...,pq_pz_cosine,variation_month,pq_z_mean_4500,variation_month_0085,pq_z_max,pq_z_min,entropy_month,std_month,diff_month,devide_month
ID,,,,,,,,,,,,,,,,,,,,,
329465205,4.0,4.0,1.0,240.0,0,6040,6040,729.000000,740.0,352.359394,...,1.054778,0.985564,0.0,0.0,1440,218,1.098612,236.742067,915.0,5.338216
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1039.318182,996.5,345.823708,...,1.048902,1.036198,0.0,0.0,2263,613,1.098612,356.826146,1349.5,6.739272
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1712.863636,1656.0,434.369417,...,0.028826,1.653016,0.0,0.0,3401,1186,0.678523,941.880880,3151.0,0.000000
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1080.636364,1068.0,341.019975,...,0.856495,1.078517,0.0,0.0,2240,553,1.098612,384.279868,1495.5,7.056769
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1180.045455,1159.5,273.560036,...,0.000000,1.732051,0.0,0.0,2211,678,0.636514,681.299561,2319.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1609.272727,1584.0,458.106474,...,1.074872,0.991196,0.0,0.0,3044,808,1.098612,532.213416,2065.5,5.876878
2864274650,13.0,13.0,0.0,120.0,0,758,758,424.545455,98.5,451.592392,...,1.128561,0.998110,0.0,0.0,1670,53,1.098612,148.889645,122.0,5.550620
2872313739,39.0,39.0,0.0,120.0,0,743,743,0.636364,0.0,2.258222,...,1.000000,1.055087,0.0,0.0,10,0,0.078869,0.329195,0.0,0.000000


<h4>tsfresh特征工程</h4>

In [24]:
month_ts_features = extract_features(
    timeseries_container=elec_month,
    default_fc_parameters=EfficientFCParameters(),
    column_id="id",
    column_sort="ym",
    profile=True,
    n_jobs=12
).fillna(0)
month_ts_features

Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 60/60 [03:35<00:00,  3.59s/it]


variable,pq_f__abs_energy,pq_f__absolute_sum_of_changes,"pq_f__agg_autocorrelation__f_agg_""mean""__maxlag_40","pq_f__agg_autocorrelation__f_agg_""median""__maxlag_40","pq_f__agg_autocorrelation__f_agg_""var""__maxlag_40","pq_f__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""","pq_f__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","pq_f__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","pq_f__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","pq_f__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""max""",...,pq_z__symmetry_looking__r_0.9500000000000001,pq_z__time_reversal_asymmetry_statistic__lag_1,pq_z__time_reversal_asymmetry_statistic__lag_2,pq_z__time_reversal_asymmetry_statistic__lag_3,pq_z__value_count__value_-1,pq_z__value_count__value_0,pq_z__value_count__value_1,pq_z__variance,pq_z__variance_larger_than_standard_deviation,pq_z__variation_coefficient
id,,,,,,,,,,,,,,,,,,,,,
329449398,1247066.0,979.0,-0.083536,-0.156298,0.079255,423.000000,266.800000,161.333333,6686.213333,0.0,...,1.0,-2.256774e+06,-1.420798e+06,-5.720574e+06,0.0,0.0,0.0,17115.307851,1.0,0.385141
329450189,3982795.0,3853.0,-0.095230,-0.043058,0.090827,986.833333,424.383333,209.166667,46502.221667,0.0,...,1.0,-6.104387e+06,-8.492686e+07,-7.885296e+07,0.0,0.0,0.0,72724.628099,1.0,0.481484
329450568,779.0,82.0,-0.118080,-0.178332,0.078580,19.500000,3.983333,0.000000,42.928333,0.0,...,1.0,3.825000e+02,6.233333e+01,0.000000e+00,0.0,17.0,1.0,122.355372,1.0,2.172782
329450586,270358.0,274.0,-0.043218,-0.022059,0.008809,180.833333,113.633333,93.333333,609.446667,0.0,...,1.0,-1.062491e+05,-1.215841e+05,-1.256302e+05,0.0,0.0,0.0,1073.572314,1.0,0.183934
329450735,3059.0,101.0,-0.056244,-0.079472,0.025958,35.833333,9.216667,4.000000,86.488333,0.0,...,1.0,7.460000e+01,3.284444e+02,1.778125e+03,0.0,0.0,0.0,108.504132,1.0,0.725202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2877042216,394.0,71.0,-0.058382,-0.100938,0.127008,8.500000,2.333333,0.166667,9.066667,0.0,...,1.0,-4.625000e+01,7.927778e+01,6.756250e+01,0.0,6.0,5.0,27.082645,1.0,1.331281
2877193822,821309.0,1470.0,-0.154513,-0.338010,0.360545,391.666667,144.050000,4.000000,13966.855000,0.0,...,1.0,1.400419e+06,6.301880e+06,6.068220e+06,0.0,0.0,0.0,16512.991736,1.0,0.597435
2877691819,23987047.0,4675.0,-0.437348,-0.611200,0.593224,1325.500000,360.100000,116.500000,126709.603333,0.0,...,1.0,4.097419e+08,1.719549e+09,2.210136e+09,0.0,0.0,0.0,752736.818182,1.0,0.832633


<h4>合并特征</h4>

In [25]:
cons_info = cons_info.merge(month_ts_features, left_index=True, right_index=True)
cons_info

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,pq_z_mean,pq_z_median,pq_z_std,...,pq_z__symmetry_looking__r_0.9500000000000001,pq_z__time_reversal_asymmetry_statistic__lag_1,pq_z__time_reversal_asymmetry_statistic__lag_2,pq_z__time_reversal_asymmetry_statistic__lag_3,pq_z__value_count__value_-1,pq_z__value_count__value_0,pq_z__value_count__value_1,pq_z__variance,pq_z__variance_larger_than_standard_deviation,pq_z__variation_coefficient
329465205,4.0,4.0,1.0,240.0,0,6040,6040,729.000000,740.0,352.359394,...,1.0,2.851816e+07,1.173997e+08,1.497055e+08,0.0,0.0,0.0,118513.636364,1.0,0.472233
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1039.318182,996.5,345.823708,...,1.0,8.036515e+07,1.646097e+08,2.601262e+08,0.0,0.0,0.0,114157.944215,1.0,0.325091
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1712.863636,1656.0,434.369417,...,1.0,1.732232e+08,4.376937e+08,6.752506e+08,0.0,0.0,0.0,180100.572314,1.0,0.247762
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1080.636364,1068.0,341.019975,...,1.0,4.545223e+07,1.570285e+08,2.541855e+08,0.0,0.0,0.0,111008.504132,1.0,0.308318
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1180.045455,1159.5,273.560036,...,1.0,1.898738e+07,5.439247e+07,1.192362e+07,0.0,0.0,0.0,71433.497934,1.0,0.226492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1609.272727,1584.0,458.106474,...,1.0,1.660451e+08,2.222076e+08,4.065167e+08,0.0,0.0,0.0,200322.380165,1.0,0.278122
2864274650,13.0,13.0,0.0,120.0,0,758,758,424.545455,98.5,451.592392,...,1.0,-1.013077e+06,-1.958095e+06,8.514425e+06,0.0,0.0,0.0,194665.884298,1.0,1.039252
2872313739,39.0,39.0,0.0,120.0,0,743,743,0.636364,0.0,2.258222,...,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,20.0,0.0,4.867769,1.0,3.467046
2872314719,39.0,39.0,0.0,120.0,0,741,741,235.090909,181.5,213.891808,...,1.0,1.715895e+05,5.947878e+06,-1.113938e+07,0.0,0.0,0.0,43670.173554,1.0,0.888908


<h3>elec_day数据预处理</h3>

<h4>读取数据并预处理</h4>

In [26]:
cons_info["kwh_mean"] = 0
cons_info["kwh_median"] = 0
cons_info["kwh_std"] = 0
cons_info["kwh_std/mean"] = 0
cons_info["kwh_entropy"] = 0

cons_info["kwh_pap_r23_distance"] = 0
cons_info["kwh_pap_r24_distance"] = 0
cons_info["kwh_pap_r34_distance"] = 0

cons_info["kwh_pap_r23_cosine"] = 0
cons_info["kwh_pap_r24_cosine"] = 0
cons_info["kwh_pap_r34_cosine"] = 0

cons_info["kwh_pap_r1_cosine"] = 0
cons_info["kwh_pap_r2_cosine"] = 0
cons_info["kwh_pap_r3_cosine"] = 0
cons_info["kwh_pap_r4_cosine"] = 0
cons_info["variation_day"] = 0

cons_info["kwh_max"] = 0
cons_info["kwh_min"] = 0

cons_info["entropy_day"] = 0
cons_info["std_day"] = 0

cons_info["diff_day"] = 0
cons_info["devide_day"] = 0

In [27]:
elec_day = pd.read_csv("../data/训练组_比特币挖矿_日用电明细（20211217）.csv")
elec_day

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,329449398,2020-01-22 00:00:00,11.77,0.0,0.0,7.40,0.0,4.36
1,329449398,2020-01-23 00:00:00,16.54,0.0,0.0,8.68,0.0,7.86
2,329449398,2020-01-24 00:00:00,10.86,0.0,0.0,5.51,0.0,5.36
3,329449398,2020-01-25 00:00:00,10.91,0.0,0.0,6.48,0.0,4.43
4,329449398,2020-01-26 00:00:00,10.85,0.0,0.0,6.32,0.0,4.52
...,...,...,...,...,...,...,...,...
954564,2878334859,2021-10-05 00:00:00,7.22,NaN,0.0,4.49,0.0,2.73
954565,2878334859,2021-10-06 00:00:00,7.23,NaN,0.0,4.48,0.0,2.74
954566,2878334859,2021-10-07 00:00:00,7.20,NaN,0.0,4.48,0.0,2.73
954567,2878334859,2021-10-08 00:00:00,7.23,NaN,0.0,4.50,0.0,2.73


In [28]:
elec_day["rq"] = pd.to_datetime(elec_day["rq"])
elec_day = elec_day.groupby('id').apply(lambda x: x.sort_values('rq', ascending=True).fillna(method="ffill").fillna(method="bfill").fillna(0)).set_index(elec_day.index)
elec_day

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,329449398,2020-01-22,11.77,0.0,0.0,7.40,0.0,4.36
1,329449398,2020-01-23,16.54,0.0,0.0,8.68,0.0,7.86
2,329449398,2020-01-24,10.86,0.0,0.0,5.51,0.0,5.36
3,329449398,2020-01-25,10.91,0.0,0.0,6.48,0.0,4.43
4,329449398,2020-01-26,10.85,0.0,0.0,6.32,0.0,4.52
...,...,...,...,...,...,...,...,...
954564,2878334859,2021-10-05,7.22,0.0,0.0,4.49,0.0,2.73
954565,2878334859,2021-10-06,7.23,0.0,0.0,4.48,0.0,2.74
954566,2878334859,2021-10-07,7.20,0.0,0.0,4.48,0.0,2.73
954567,2878334859,2021-10-08,7.23,0.0,0.0,4.50,0.0,2.73


<h4>手工特征工程</h4>

In [29]:
for cons_id in tqdm(cons_info.index):
    df = elec_day[elec_day["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("rq").set_index("rq")
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    
    cons_info.loc[cons_id, "kwh_mean"] = df["kwh"].mean()
    cons_info.loc[cons_id, "kwh_median"] = df["kwh"].median()
    cons_info.loc[cons_id, "kwh_std"] = df["kwh"].std()
    cons_info.loc[(cons_info.index == cons_id) & (cons_info["kwh_mean"] > 0), "kwh_std/mean"] = cons_info["kwh_std"] / cons_info["kwh_mean"]
    cons_info.loc[cons_id, "kwh_entropy"] = entropy(df["kwh"])

    cons_info.loc[cons_id, "kwh_pap_r23_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info.loc[cons_id, "kwh_pap_r24_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info.loc[cons_id, "kwh_pap_r34_distance"] = distance.euclidean(df["kwh_pap_r3"], df["kwh_pap_r4"])

    cons_info.loc[cons_id, "kwh_pap_r23_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info.loc[cons_id, "kwh_pap_r24_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info.loc[cons_id, "kwh_pap_r34_cosine"] = cosine(df["kwh_pap_r3"], df["kwh_pap_r4"])
    
    cons_info.loc[cons_id, "kwh_pap_r1_cosine"] = cosine(df["kwh_pap_r1"], df["kwh"])
    cons_info.loc[cons_id, "kwh_pap_r2_cosine"] = cosine(df["kwh_pap_r2"], df["kwh"])
    cons_info.loc[cons_id, "kwh_pap_r3_cosine"] = cosine(df["kwh_pap_r3"], df["kwh"])
    cons_info.loc[cons_id, "kwh_pap_r4_cosine"] = cosine(df["kwh_pap_r4"], df["kwh"])
    
    cons_info.loc[cons_id, "variation_day"] = variation(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    cons_info.loc[cons_id, "kwh_max"] = df["kwh"].max()
    cons_info.loc[cons_id, "kwh_min"] = df["kwh"].min()
    
    cons_info.loc[cons_id, "entropy_day"] = entropy_df(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    cons_info.loc[cons_id, "std_day"] = std_df(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    cons_info.loc[cons_id, "diff_day"] = diff_median(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    cons_info.loc[cons_id, "devide_day"] = devide_median(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    
cons_info

100%|████████████████████████████████████████████████████████████████████████████████| 8840/8840 [04:45<00:00, 30.91it/s]


,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,pq_z_mean,pq_z_median,pq_z_std,...,kwh_pap_r2_cosine,kwh_pap_r3_cosine,kwh_pap_r4_cosine,variation_day,kwh_max,kwh_min,entropy_day,std_day,diff_day,devide_day
329465205,4.0,4.0,1.0,240.0,0,6040,6040,729.000000,740.0,352.359394,...,0.013884,1.0,0.048088,1.039291,45.71,0.66,1.060104,6.192106,20.875,3.454500e+08
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1039.318182,996.5,345.823708,...,0.004351,1.0,0.019658,1.045283,54.73,17.88,1.098612,12.109005,47.185,7.074000e+08
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1712.863636,1656.0,434.369417,...,0.001687,1.0,0.024876,1.236592,79.75,26.31,1.090055,22.801373,88.530,1.329000e+09
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1080.636364,1068.0,341.019975,...,0.002899,1.0,0.017558,1.103814,74.44,14.16,1.098612,12.624684,49.825,7.327500e+08
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1180.045455,1159.5,273.560036,...,0.003703,1.0,0.023050,1.094278,52.60,16.35,1.098612,12.514458,47.980,7.183500e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1609.272727,1584.0,458.106474,...,0.005853,1.0,0.019615,0.962473,74.60,13.68,1.085776,12.263838,42.910,6.667500e+08
2864274650,13.0,13.0,0.0,120.0,0,758,758,424.545455,98.5,451.592392,...,0.002854,1.0,0.013023,0.965715,42.91,1.50,1.085776,4.196080,4.035,5.565000e+07
2872313739,39.0,39.0,0.0,120.0,0,743,743,0.636364,0.0,2.258222,...,0.000000,0.0,0.000000,1.000000,0.00,0.00,0.000000,0.000000,0.000,0.000000e+00
2872314719,39.0,39.0,0.0,120.0,0,741,741,235.090909,181.5,213.891808,...,0.011304,1.0,0.135482,1.030999,60.99,4.68,1.034432,2.761213,7.980,1.218000e+08


<h4>tsfresh特征工程</h4>

In [30]:
day_ts_features = extract_features(
    timeseries_container=elec_day,
    default_fc_parameters=EfficientFCParameters(),
    column_id="id",
    column_sort="rq",
    n_jobs=12
).fillna(0)
day_ts_features

Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 60/60 [06:29<00:00,  6.49s/it]


variable,kwh__abs_energy,kwh__absolute_sum_of_changes,"kwh__agg_autocorrelation__f_agg_""mean""__maxlag_40","kwh__agg_autocorrelation__f_agg_""median""__maxlag_40","kwh__agg_autocorrelation__f_agg_""var""__maxlag_40","kwh__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""","kwh__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","kwh__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","kwh__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","kwh__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""max""",...,kwh_rap__symmetry_looking__r_0.9500000000000001,kwh_rap__time_reversal_asymmetry_statistic__lag_1,kwh_rap__time_reversal_asymmetry_statistic__lag_2,kwh_rap__time_reversal_asymmetry_statistic__lag_3,kwh_rap__value_count__value_-1,kwh_rap__value_count__value_0,kwh_rap__value_count__value_1,kwh_rap__variance,kwh_rap__variance_larger_than_standard_deviation,kwh_rap__variation_coefficient
id,,,,,,,,,,,,,,,,,,,,,
329449398,16534.8711,253.10,0.106117,0.120836,0.038563,20.214091,14.880682,11.593636,18.910369,37.351667,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
329450189,105969.2063,928.99,-0.028513,-0.097598,0.053594,67.754545,33.789375,9.085455,618.418485,104.795000,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
329450568,198.0248,16.59,-0.013877,-0.130050,0.080029,2.755909,1.347500,0.000000,2.127854,4.260000,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
329450586,3444.6803,43.85,0.032350,0.006470,0.020351,6.925455,5.944830,5.059091,0.382722,7.608333,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
329450735,21.9048,12.79,0.049677,0.029483,0.012682,0.416818,0.231295,0.139091,0.014717,1.321667,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2877042216,34.0548,13.48,0.065788,-0.030742,0.037502,1.218182,0.528716,0.000000,0.412552,2.371667,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
2877193822,5356.5803,232.45,-0.027226,-0.077987,0.080047,7.948182,3.132807,1.111818,9.057177,23.610000,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
2877691819,238786.6131,886.49,0.378638,0.319650,0.064073,0.270000,-3.850670,-2.194545,-49.171118,66.135000,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0


<h4>特征合并</h4>

In [31]:
cons_info = cons_info.merge(day_ts_features, left_index=True, right_index=True)
cons_info

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,IS_FLAG,live_days,check_days,pq_z_mean,pq_z_median,pq_z_std,...,kwh_rap__symmetry_looking__r_0.9500000000000001,kwh_rap__time_reversal_asymmetry_statistic__lag_1,kwh_rap__time_reversal_asymmetry_statistic__lag_2,kwh_rap__time_reversal_asymmetry_statistic__lag_3,kwh_rap__value_count__value_-1,kwh_rap__value_count__value_0,kwh_rap__value_count__value_1,kwh_rap__variance,kwh_rap__variance_larger_than_standard_deviation,kwh_rap__variation_coefficient
329465205,4.0,4.0,1.0,240.0,0,6040,6040,729.000000,740.0,352.359394,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
329465245,4.0,4.0,1.0,240.0,0,6040,6040,1039.318182,996.5,345.823708,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
329465357,8.0,8.0,1.0,240.0,0,6040,6040,1712.863636,1656.0,434.369417,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
329465360,2.0,2.0,1.0,240.0,0,6040,6040,1080.636364,1068.0,341.019975,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
329465439,8.0,8.0,1.0,240.0,0,6040,6040,1180.045455,1159.5,273.560036,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862711473,39.0,39.0,0.0,120.0,0,759,759,1609.272727,1584.0,458.106474,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
2864274650,13.0,13.0,0.0,120.0,0,758,758,424.545455,98.5,451.592392,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
2872313739,39.0,39.0,0.0,120.0,0,743,743,0.636364,0.0,2.258222,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
2872314719,39.0,39.0,0.0,120.0,0,741,741,235.090909,181.5,213.891808,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0


<h2>数据集划分</h2>

In [32]:
x = cons_info.drop(columns=["IS_FLAG"])
y = cons_info["IS_FLAG"]
print(x.shape, y.shape)

(8840, 7621) (8840,)


<h2>标准化</h2>

In [33]:
ss = StandardScaler()
x = ss.fit_transform(x)
x

array([[-0.31260749, -0.31917163,  1.05812375, ..., -0.01594378,
        -0.05107443, -0.06690109],
       [-0.31260749, -0.31917163,  1.05812375, ..., -0.01594378,
        -0.05107443, -0.06690109],
       [-0.2464131 , -0.25086739,  1.05812375, ..., -0.01594378,
        -0.05107443, -0.06690109],
       ...,
       [ 0.2665934 ,  0.27849043, -0.86811471, ..., -0.01594378,
        -0.05107443, -0.06690109],
       [ 0.2665934 ,  0.27849043, -0.86811471, ..., -0.01594378,
        -0.05107443, -0.06690109],
       [ 0.11765603,  0.1248059 , -0.86811471, ..., -0.01594378,
        -0.05107443, -0.06690109]])

<h1>做出来一个模型</h1>

In [34]:
smo = ADASYN(random_state=42, n_jobs=-1)
x, y = smo.fit_sample(x, y)

In [35]:
model = LGBMClassifier(
    objective="binary", 
    learning_rate=0.01,
    n_jobs=-1
)

In [36]:
feature_sel = SelectFromModel(
    estimator=model,
    threshold=1,
    max_features=100
)
x = feature_sel.fit_transform(x, y)

In [37]:
cv = GridSearchCV(
    estimator=model,
    param_grid={
        "n_estimators": list(range(50, 700, 50)),
        "num_leaves": list(range(1, 50, 5)),
        "max_depth": [-1] + list(range(2, 11, 2)),
        "subsample_for_bin": list(range(100000, 700000, 100000))
    },
    scoring="f1",
    n_jobs=-1,
    iid=False,
    cv=10,
    verbose=1
)

In [38]:
try:
    model = load("../models/best_lightBGM.pkl")
    print("n_estimators", model.n_estimators)
    print("num_leaves", model.num_leaves)
    print("max_depth", model.max_depth)
    print("subsample_for_bin", model.subsample_for_bin)
    model.fit(x, y)
    f1 = f1_score(y, model.predict(x))
except FileNotFoundError as e:
    cv.fit(x, y)
    model = cv.best_estimator_
    print(cv.best_params_)
    f1 = cv.best_score_
    model.fit(x, y)
    dump(model, "../models/best_lightBGM.pkl")

n_estimators 200
num_leaves 16
max_depth 6
subsample_for_bin 100000


In [39]:
f1

1.0

In [40]:
accuracy_score(y, model.predict(x))

1.0

In [41]:
roc_auc_score(y, model.predict(x))

1.0

In [42]:
features = list(cons_info.columns)
features.pop(features.index("IS_FLAG"))
for i, importance in enumerate(model.feature_importances_):
    print(features[i] + "\t" + str(importance))

CONTRACT_CAP	26
RUN_CAP	8
SHIFT_NO	18
CHK_CYCLE	94
live_days	5
check_days	44
pq_z_mean	9
pq_z_median	3
pq_z_std	12
pq_z_std/mean	0
pq_z_entropy	12
pq_fg_distance	1
pq_gp_diatance	19
pq_fp_distance	13
pq_fg_cosine	60
pq_gp_cosine	2
pq_fp_cosine	5
pq_fz_cosine	9
pq_gz_cosine	2
pq_pz_cosine	17
variation_month	5
pq_z_mean_4500	3
variation_month_0085	113
pq_z_max	1
pq_z_min	17
entropy_month	3
std_month	102
diff_month	2
devide_month	60
pq_f__abs_energy	51
pq_f__absolute_sum_of_changes	14
pq_f__agg_autocorrelation__f_agg_"mean"__maxlag_40	2
pq_f__agg_autocorrelation__f_agg_"median"__maxlag_40	53
pq_f__agg_autocorrelation__f_agg_"var"__maxlag_40	103
pq_f__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"max"	2
pq_f__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"mean"	17
pq_f__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"min"	9
pq_f__agg_linear_trend__attr_"intercept"__chunk_len_10__f_agg_"var"	5
pq_f__agg_linear_trend__attr_"intercept"__chunk_len_50__f_agg_"max"	

<h1>处理测试数据</h1>

<h2>数据预处理</h2>

<h3>档案数据预处理</h3>

In [43]:
cons_info_test = pd.read_csv("../data/测试组_比特币挖矿_档案明细（20211220）.csv", index_col=0, quotechar='"', encoding="gbk")
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CANCEL_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,TMP_DATE
ID,,,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998/12/1,NaN,24,1998/12/1,非临时用电,NaN
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,1998/12/1,NaN,24,1998/12/1,非临时用电,NaN
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999/4/1,NaN,24,1999/4/1,非临时用电,NaN
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999/3/1,NaN,36,1999/3/1,非临时用电,NaN
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998/12/1,NaN,24,1998/12/1,非临时用电,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,2019/11/8 12:42:00,NaN,120,2019/11/8 12:42:00,非临时用电,NaN
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,NaN,2019/11/18 14:08:00,NaN,120,2019/11/18 14:08:00,非临时用电,NaN
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,NaN,2019/11/18 14:08:00,NaN,120,2019/11/18 14:08:00,非临时用电,NaN


In [44]:
cons_info_test = cons_info_test.drop(columns=["CANCEL_DATE", "TMP_DATE"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME
ID,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998/12/1,24,1998/12/1,非临时用电
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,1998/12/1,24,1998/12/1,非临时用电
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999/4/1,24,1999/4/1,非临时用电
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999/3/1,36,1999/3/1,非临时用电
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998/12/1,24,1998/12/1,非临时用电
...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,2019/11/8 12:42:00,120,2019/11/8 12:42:00,非临时用电
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,NaN,2019/11/18 14:08:00,120,2019/11/18 14:08:00,非临时用电
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,NaN,2019/11/18 14:08:00,120,2019/11/18 14:08:00,非临时用电


In [45]:
cons_info_test["BUILD_DATE"] = pd.to_datetime(cons_info_test["BUILD_DATE"])
cons_info_test["LAST_CHK_DATE"] = pd.to_datetime(cons_info_test["LAST_CHK_DATE"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME
ID,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999-04-01 00:00:00,24,1999-04-01 00:00:00,非临时用电
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999-03-01 00:00:00,36,1999-03-01 00:00:00,非临时用电
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,NaN,2019-11-08 12:42:00,120,2019-11-08 12:42:00,非临时用电
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,NaN,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,NaN,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电


In [46]:
cons_info_test["SHIFT_NO"] = cons_info_test["SHIFT_NO"].fillna(0)
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME
ID,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999-04-01 00:00:00,24,1999-04-01 00:00:00,非临时用电
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999-03-01 00:00:00,36,1999-03-01 00:00:00,非临时用电
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电
...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,2019-11-08 12:42:00,120,2019-11-08 12:42:00,非临时用电
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电


In [47]:
cons_info_test["now"] = "2021-12-31"
cons_info_test["now"] = pd.to_datetime(cons_info_test["now"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,now
ID,,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999-04-01 00:00:00,24,1999-04-01 00:00:00,非临时用电,2021-12-31
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999-03-01 00:00:00,36,1999-03-01 00:00:00,非临时用电,2021-12-31
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31
...,...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,2019-11-08 12:42:00,120,2019-11-08 12:42:00,非临时用电,2021-12-31
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电,2021-12-31
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电,2021-12-31


In [48]:
cons_info_test["live_days"] = (cons_info_test["now"] - cons_info_test["BUILD_DATE"]).map(lambda x: x.days)
cons_info_test["check_days"] = (cons_info_test["now"] - cons_info_test["LAST_CHK_DATE"]).map(lambda x: x.days)
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,BUILD_DATE,CHK_CYCLE,LAST_CHK_DATE,TMP_NAME,now,live_days,check_days
ID,,,,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31,8431,8431
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31,8431,8431
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1999-04-01 00:00:00,24,1999-04-01 00:00:00,非临时用电,2021-12-31,8310,8310
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,1999-03-01 00:00:00,36,1999-03-01 00:00:00,非临时用电,2021-12-31,8341,8341
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,1998-12-01 00:00:00,24,1998-12-01 00:00:00,非临时用电,2021-12-31,8431,8431
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,2019-11-08 12:42:00,120,2019-11-08 12:42:00,非临时用电,2021-12-31,783,783
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电,2021-12-31,773,773
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,0.0,2019-11-18 14:08:00,120,2019-11-18 14:08:00,非临时用电,2021-12-31,773,773


In [49]:
cons_info_test = cons_info_test.drop(columns=["BUILD_DATE", "LAST_CHK_DATE", "now"])
cons_info_test

,ELEC_TYPE_NAME,VOLT_NAME,PRC_NAME,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,TMP_NAME,live_days,check_days
ID,,,,,,,,,,
179406029,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,24,非临时用电,8431,8431
179406030,城镇居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,24,非临时用电,8431,8431
179406094,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,24,非临时用电,8310,8310
179406097,城镇居民生活用电,交流220V,居民生活<1kV(合表),4.0,4.0,1.0,36,非临时用电,8341,8341
179406099,城镇居民生活用电,交流220V,居民生活<1kV(合表),2.0,2.0,1.0,24,非临时用电,8431,8431
...,...,...,...,...,...,...,...,...,...,...
2849971818,乡村居民生活用电,交流220V,居民生活<1kV(合表),12.0,12.0,0.0,120,非临时用电,783,783
2850017469,乡村居民生活用电,交流380V,居民生活<1kV(合表),120.0,120.0,0.0,120,非临时用电,773,773
2850017472,乡村居民生活用电,交流380V,居民生活<1kV(合表),720.0,720.0,0.0,120,非临时用电,773,773


In [50]:
cons_info_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15379 entries, 179406029 to 2852368013
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ELEC_TYPE_NAME  15379 non-null  object 
 1   VOLT_NAME       15379 non-null  object 
 2   PRC_NAME        15379 non-null  object 
 3   CONTRACT_CAP    15379 non-null  float64
 4   RUN_CAP         15379 non-null  float64
 5   SHIFT_NO        15379 non-null  float64
 6   CHK_CYCLE       15379 non-null  int64  
 7   TMP_NAME        15379 non-null  object 
 8   live_days       15379 non-null  int64  
 9   check_days      15379 non-null  int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 1.3+ MB


In [51]:
# 因为看过了这部分特征重要性为0，所以丢掉
cons_info_test = cons_info_test.drop(columns=["ELEC_TYPE_NAME", "VOLT_NAME", "PRC_NAME", "TMP_NAME"])
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days
ID,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431
179406030,12.0,12.0,0.0,24,8431,8431
179406094,2.0,2.0,1.0,24,8310,8310
179406097,4.0,4.0,1.0,36,8341,8341
179406099,2.0,2.0,1.0,24,8431,8431
...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783
2850017469,120.0,120.0,0.0,120,773,773
2850017472,720.0,720.0,0.0,120,773,773


In [52]:
cons_info_test = pd.get_dummies(cons_info_test)
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days
ID,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431
179406030,12.0,12.0,0.0,24,8431,8431
179406094,2.0,2.0,1.0,24,8310,8310
179406097,4.0,4.0,1.0,36,8341,8341
179406099,2.0,2.0,1.0,24,8431,8431
...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783
2850017469,120.0,120.0,0.0,120,773,773
2850017472,720.0,720.0,0.0,120,773,773


In [53]:
cons_info_test["CHK_CYCLE"] = cons_info_test["CHK_CYCLE"].fillna(cons_info_test["CHK_CYCLE"].median())
cons_info_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15379 entries, 179406029 to 2852368013
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CONTRACT_CAP  15379 non-null  float64
 1   RUN_CAP       15379 non-null  float64
 2   SHIFT_NO      15379 non-null  float64
 3   CHK_CYCLE     15379 non-null  int64  
 4   live_days     15379 non-null  int64  
 5   check_days    15379 non-null  int64  
dtypes: float64(3), int64(3)
memory usage: 841.0 KB


cons_info_test["PRC_NAME_居民合表电价(1-10千伏）"] = cons_info_test["PRC_NAME_居民合表电价(1-10千伏）"] + cons_info_test["PRC_NAME_居民生活1_10kV(合表)"]
cons_info_test = cons_info_test.drop(columns=["PRC_NAME_居民生活1_10kV(合表)"])
cons_info_test

cons_info_test = cons_info_test.rename(columns={"PRC_NAME_居民生活<1kV(合表)": "PRC_NAME_居民合表电价(不满1千伏）"})
cons_info_test

cons_info_test["VOLT_NAME_交流10kV"] = cons_info_test["VOLT_NAME_交流10kV"] + cons_info_test["VOLT_NAME_交流6kV"]
cons_info_test = cons_info_test.drop(columns=["VOLT_NAME_交流6kV"])
cons_info_test

cons_info_test["ELEC_TYPE_NAME_普通工业"] = cons_info_test["ELEC_TYPE_NAME_普通工业"] + cons_info_test["ELEC_TYPE_NAME_大工业用电"]
cons_info_test = cons_info_test.drop(columns=["ELEC_TYPE_NAME_大工业用电"])
cons_info_test["ELEC_TYPE_NAME_非工业"] = cons_info_test["ELEC_TYPE_NAME_非工业"] + cons_info_test["ELEC_TYPE_NAME_农业生产用电"]
cons_info_test = cons_info_test.drop(columns=["ELEC_TYPE_NAME_农业生产用电"])
cons_info_test

<h3>月电量处理</h3>

<h4>读取数据并预处理</h4>

In [54]:
cons_info_test["pq_z_mean"] = 0
cons_info_test["pq_z_median"] = 0
cons_info_test["pq_z_std"] = 0
cons_info_test["pq_z_std/mean"] = 0
cons_info_test["pq_z_entropy"] = 0
cons_info_test["pq_fg_distance"] = 0
cons_info_test["pq_gp_diatance"] = 0
cons_info_test["pq_fp_distance"] = 0
cons_info_test["pq_fg_cosine"] = 0
cons_info_test["pq_gp_cosine"] = 0
cons_info_test["pq_fp_cosine"] = 0
cons_info_test["pq_fz_cosine"] = 0
cons_info_test["pq_gz_cosine"] = 0
cons_info_test["pq_pz_cosine"] = 0
cons_info_test["variation_month"] = 0
cons_info_test["pq_z_mean_4500"] = 0
cons_info_test["variation_month_0085"] = 0

cons_info_test["pq_z_max"] = 0
cons_info_test["pq_z_min"] = 0

cons_info_test["entropy_month"] = 0
cons_info_test["std_month"] = 0

cons_info_test["diff_month"] = 0
cons_info_test["devide_month"] = 0

In [55]:
elec_month_test = pd.read_csv("../data/测试组_比特币挖矿_月用电明细（20211217）.csv")
elec_month_test

,id,ym,pq_f,pq_g,pq_p,pq_z
0,179438260,202001,244,111,1,356
1,179438260,202002,217,108,-1,324
2,179438260,202003,280,90,0,370
3,179438260,202004,338,118,1,457
4,179438260,202005,297,120,-1,416
...,...,...,...,...,...,...
338333,2880712108,202106,474,675,0,1149
338334,2880712108,202107,342,881,0,1223
338335,2880712108,202108,293,772,0,1065
338336,2880712108,202109,418,570,-1,987


In [56]:
elec_month_test["ym"] = elec_month_test["ym"].astype(str)
elec_month_test = elec_month_test.groupby('id').apply(lambda x: x.sort_values('ym', ascending=True).fillna(method="ffill").fillna(method="bfill").fillna(0)).set_index(elec_month_test.index)
elec_month_test

,id,ym,pq_f,pq_g,pq_p,pq_z
0,179404030,202001,487,215,-1,701
1,179404030,202002,736,416,0,1152
2,179404030,202003,699,477,1,1177
3,179404030,202004,1122,688,-1,1809
4,179404030,202005,564,452,0,1016
...,...,...,...,...,...,...
338333,2880712108,202106,474,675,0,1149
338334,2880712108,202107,342,881,0,1223
338335,2880712108,202108,293,772,0,1065
338336,2880712108,202109,418,570,-1,987


<h4>手工特征工程</h4>

In [57]:
for cons_id in tqdm(cons_info_test.index):
    df = elec_month_test[elec_month_test["id"] == cons_id]
    df = df.drop(columns=["id"]).sort_values("ym").set_index("ym")
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    
    cons_info_test.loc[cons_id, "pq_z_mean"] = df["pq_z"].mean()
    cons_info_test.loc[cons_id, "pq_z_median"] = df["pq_z"].median()
    cons_info_test.loc[cons_id, "pq_z_std"] = df["pq_z"].std()
    cons_info_test.loc[(cons_info_test.index == cons_id) & (cons_info_test["pq_z_mean"] > 0), "pq_z_std/mean"] = cons_info_test["pq_z_std"] / cons_info_test["pq_z_mean"]
    cons_info_test.loc[cons_id, "pq_z_entropy"] = entropy(df["pq_z"])
    
    cons_info_test.loc[cons_id, "pq_fg_distance"] = distance.euclidean(df["pq_f"], df["pq_g"])
    cons_info_test.loc[cons_id, "pq_gp_diatance"] = distance.euclidean(df["pq_g"], df["pq_p"])
    cons_info_test.loc[cons_id, "pq_fp_distance"] = distance.euclidean(df["pq_f"], df["pq_p"])
    
    cons_info_test.loc[cons_id, "pq_fg_cosine"] = cosine(df["pq_f"], df["pq_g"])
    cons_info_test.loc[cons_id, "pq_gp_cosine"] = cosine(df["pq_g"], df["pq_p"])
    cons_info_test.loc[cons_id, "pq_fp_cosine"] = cosine(df["pq_f"], df["pq_p"])
    cons_info_test.loc[cons_id, "pq_fz_cosine"] = cosine(df["pq_f"], df["pq_z"])
    cons_info_test.loc[cons_id, "pq_gz_cosine"] = cosine(df["pq_g"], df["pq_z"])
    cons_info_test.loc[cons_id, "pq_pz_cosine"] = cosine(df["pq_p"], df["pq_z"])
    
    cons_info_test.loc[cons_id, "variation_month"] = variation(df[["pq_f", "pq_g", "pq_p"]])
    cons_info_test.loc[cons_id, "pq_z_mean_4500"] = (cons_info_test.loc[cons_id, "pq_z_mean"] > 4500).astype(float)
    cons_info_test.loc[cons_id, "variation_month_0085"] = (cons_info_test.loc[cons_id, "variation_month"] < 0.085).astype(float)

    cons_info_test.loc[cons_id, "pq_z_max"] = df["pq_z"].max()
    cons_info_test.loc[cons_id, "pq_z_min"] = df["pq_z"].min()
    
    cons_info_test.loc[cons_id, "entropy_month"] = entropy_df(df[["pq_f", "pq_g", "pq_p"]])
    cons_info_test.loc[cons_id, "std_month"] = std_df(df[["pq_f", "pq_g", "pq_p"]])
    
    cons_info_test.loc[cons_id, "diff_month"] = diff_median(df[["pq_f", "pq_g", "pq_p"]])
    cons_info_test.loc[cons_id, "devide_month"] = devide_median(df[["pq_f", "pq_g", "pq_p"]])  
    
    
cons_info_test

100%|██████████████████████████████████████████████████████████████████████████████| 15379/15379 [05:00<00:00, 51.20it/s]


,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,pq_z_mean,pq_z_median,pq_z_std,pq_z_std/mean,...,pq_pz_cosine,variation_month,pq_z_mean_4500,variation_month_0085,pq_z_max,pq_z_min,entropy_month,std_month,diff_month,devide_month
ID,,,,,,,,,,,,,,,,,,,,,
179406029,2.0,2.0,1.0,24,8431,8431,815.863636,737.0,213.639281,0.261857,...,0.964811,0.980133,0.0,0.0,1366,587,1.035599,262.464781,934.5,5.250582
179406030,12.0,12.0,0.0,24,8431,8431,352.409091,305.5,186.512519,0.529250,...,0.000000,1.732051,0.0,0.0,865,131,0.636514,203.463484,611.0,0.000000
179406094,2.0,2.0,1.0,24,8310,8310,590.409091,564.0,102.925150,0.174329,...,1.105591,1.496756,0.0,0.0,984,483,0.867563,294.951725,1006.5,29.204981
179406097,4.0,4.0,1.0,36,8341,8341,676.727273,637.5,250.357874,0.369954,...,1.140851,1.014585,0.0,0.0,1299,360,1.077608,221.420533,811.5,5.556705
179406099,2.0,2.0,1.0,24,8431,8431,396.772727,399.0,183.313141,0.462010,...,1.072008,0.969216,0.0,0.0,804,80,1.077608,128.638531,496.0,5.175580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,222.545455,197.0,90.766434,0.407856,...,0.853454,1.253312,0.0,0.0,554,125,1.056603,94.071420,313.5,0.684623
2850017469,120.0,120.0,0.0,120,773,773,0.000000,0.0,0.000000,0.000000,...,0.000000,1.000000,0.0,0.0,0,0,0.000000,0.000000,0.0,0.000000
2850017472,720.0,720.0,0.0,120,773,773,23660.727273,24569.0,8178.414878,0.345654,...,0.001275,0.280995,1.0,0.0,40812,0,0.880640,1957.498271,7118.0,4.984970


<h4>tsfresh特征工程</h4>

In [58]:
month_test_ts_features = extract_features(
    timeseries_container=elec_month_test,
    default_fc_parameters=EfficientFCParameters(),
    column_id="id",
    column_sort="ym",
    profile=True,
    n_jobs=12
).fillna(0)
month_test_ts_features

Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 60/60 [06:17<00:00,  6.29s/it]


variable,pq_f__abs_energy,pq_f__absolute_sum_of_changes,"pq_f__agg_autocorrelation__f_agg_""mean""__maxlag_40","pq_f__agg_autocorrelation__f_agg_""median""__maxlag_40","pq_f__agg_autocorrelation__f_agg_""var""__maxlag_40","pq_f__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""","pq_f__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","pq_f__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","pq_f__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","pq_f__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""max""",...,pq_z__symmetry_looking__r_0.9500000000000001,pq_z__time_reversal_asymmetry_statistic__lag_1,pq_z__time_reversal_asymmetry_statistic__lag_2,pq_z__time_reversal_asymmetry_statistic__lag_3,pq_z__value_count__value_-1,pq_z__value_count__value_0,pq_z__value_count__value_1,pq_z__variance,pq_z__variance_larger_than_standard_deviation,pq_z__variation_coefficient
id,,,,,,,,,,,,,,,,,,,,,
179404030,25744074.0,7828.0,-0.264622,-0.237001,0.221521,1283.666667,557.366667,-74.666667,182772.846667,0.0,...,1.0,2.632053e+09,3.280653e+09,5.016042e+09,0.0,0.0,0.0,1.017002e+06,1.0,0.633151
179404040,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,1.0,-1.161956e+07,-5.526159e+07,-6.475557e+07,0.0,0.0,0.0,4.217070e+04,1.0,0.403773
179404046,42969989.0,6543.0,-0.342414,-0.315596,0.362171,2230.166667,1026.883333,677.166667,178197.588333,0.0,...,1.0,1.107092e+09,4.375649e+09,5.672051e+09,0.0,0.0,0.0,5.612553e+05,1.0,0.285943
179404049,21263119.0,3753.0,-0.036762,0.038267,0.066126,1544.666667,988.500000,751.000000,45182.483333,0.0,...,1.0,1.087764e+08,5.008349e+08,4.578607e+08,0.0,0.0,0.0,1.218984e+05,1.0,0.229046
179404054,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,1.0,4.362587e+06,5.088970e+07,3.700435e+07,0.0,0.0,0.0,4.434475e+04,1.0,0.467818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2877297622,21599.0,289.0,-0.001403,-0.025070,0.209899,56.166667,30.750000,2.500000,203.441667,0.0,...,1.0,-5.695150e+03,-2.054806e+04,-4.102188e+03,0.0,1.0,0.0,4.892397e+02,1.0,0.482751
2878310631,198024.0,317.0,-0.444219,-0.915696,0.584685,70.166667,25.200000,-18.833333,748.946667,0.0,...,1.0,5.355725e+05,1.176498e+06,2.079852e+06,0.0,3.0,0.0,8.815994e+03,1.0,0.913604
2880078775,1544636.0,2645.0,-0.155663,-0.211903,0.104203,562.000000,169.516667,0.500000,25585.928333,0.0,...,1.0,3.672076e+07,6.460272e+07,6.188907e+07,0.0,1.0,0.0,1.741381e+05,1.0,0.625124


<h4>合并特征</h4>

In [59]:
cons_info_test = cons_info_test.merge(month_test_ts_features, left_index=True, right_index=True)
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,pq_z_mean,pq_z_median,pq_z_std,pq_z_std/mean,...,pq_z__symmetry_looking__r_0.9500000000000001,pq_z__time_reversal_asymmetry_statistic__lag_1,pq_z__time_reversal_asymmetry_statistic__lag_2,pq_z__time_reversal_asymmetry_statistic__lag_3,pq_z__value_count__value_-1,pq_z__value_count__value_0,pq_z__value_count__value_1,pq_z__variance,pq_z__variance_larger_than_standard_deviation,pq_z__variation_coefficient
179406029,2.0,2.0,1.0,24,8431,8431,815.863636,737.0,213.639281,0.261857,...,1.0,3.856715e+07,1.045349e+08,1.373023e+08,0.0,0.0,0.0,4.356712e+04,1.0,0.255836
179406030,12.0,12.0,0.0,24,8431,8431,352.409091,305.5,186.512519,0.529250,...,1.0,4.977945e+04,-8.281569e+05,-6.863715e+06,0.0,0.0,0.0,3.320570e+04,1.0,0.517082
179406094,2.0,2.0,1.0,24,8310,8310,590.409091,564.0,102.925150,0.174329,...,1.0,-6.072142e+06,-1.253744e+07,-1.723061e+07,0.0,0.0,0.0,1.011206e+04,1.0,0.170320
179406097,4.0,4.0,1.0,36,8341,8341,676.727273,637.5,250.357874,0.369954,...,1.0,4.088496e+07,7.580914e+07,9.090102e+07,0.0,0.0,0.0,5.983002e+04,1.0,0.361448
179406099,2.0,2.0,1.0,24,8431,8431,396.772727,399.0,183.313141,0.462010,...,1.0,-3.265024e+05,3.031430e+06,-7.066469e+06,0.0,0.0,0.0,3.207627e+04,1.0,0.451388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,222.545455,197.0,90.766434,0.407856,...,1.0,-9.972131e+05,-1.200872e+06,-7.520287e+05,0.0,0.0,0.0,7.864066e+03,1.0,0.398478
2850017469,120.0,120.0,0.0,120,773,773,0.000000,0.0,0.000000,0.000000,...,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.0,22.0,0.0,0.000000e+00,0.0,0.000000
2850017472,720.0,720.0,0.0,120,773,773,23660.727273,24569.0,8178.414878,0.345654,...,1.0,2.023885e+12,3.255538e+12,4.913826e+12,0.0,1.0,0.0,6.384618e+07,1.0,0.337706
2851323065,12.0,12.0,0.0,120,780,780,210.227273,167.5,217.211973,1.033225,...,1.0,2.984783e+06,2.256835e+07,2.377736e+07,0.0,7.0,0.0,4.503645e+04,1.0,1.009469


<h3>日电量处理</h3>

<h4>读取数据并预处理</h4>

In [60]:
cons_info_test["kwh_mean"] = 0
cons_info_test["kwh_median"] = 0
cons_info_test["kwh_std"] = 0
cons_info_test["kwh_std/mean"] = 0
cons_info_test["kwh_entropy"] = 0

cons_info_test["kwh_pap_r23_distance"] = 0
cons_info_test["kwh_pap_r24_distance"] = 0
cons_info_test["kwh_pap_r34_distance"] = 0

cons_info_test["kwh_pap_r23_cosine"] = 0
cons_info_test["kwh_pap_r24_cosine"] = 0
cons_info_test["kwh_pap_r34_cosine"] = 0

cons_info_test["kwh_pap_r1_cosine"] = 0
cons_info_test["kwh_pap_r2_cosine"] = 0
cons_info_test["kwh_pap_r3_cosine"] = 0
cons_info_test["kwh_pap_r4_cosine"] = 0
cons_info_test["variation_day"] = 0

cons_info_test["kwh_max"] = 0
cons_info_test["kwh_min"] = 0

cons_info_test["entropy_day"] = 0
cons_info_test["std_day"] = 0

cons_info_test["diff_day"] = 0
cons_info_test["devide_day"] = 0

In [61]:
elec_day_test = pd.read_csv("../data/测试组_比特币挖矿_日用电明细（20211217）.csv")
elec_day_test

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,179404030,2020-01-22 00:00:00,51.77,NaN,0.0,33.47,0.0,18.31
1,179404030,2020-01-23 00:00:00,73.27,NaN,0.0,51.83,0.0,21.44
2,179404030,2020-01-24 00:00:00,74.90,NaN,0.0,52.20,0.0,22.70
3,179404030,2020-01-25 00:00:00,62.73,NaN,0.0,37.27,0.0,25.45
4,179404030,2020-01-26 00:00:00,64.27,NaN,0.0,41.72,0.0,22.56
...,...,...,...,...,...,...,...,...
1660582,2880712108,2021-10-05 00:00:00,44.85,NaN,0.0,22.14,0.0,22.71
1660583,2880712108,2021-10-06 00:00:00,28.89,NaN,0.0,14.94,0.0,13.95
1660584,2880712108,2021-10-07 00:00:00,51.10,NaN,0.0,27.27,0.0,23.84
1660585,2880712108,2021-10-08 00:00:00,17.96,NaN,0.0,7.19,0.0,10.76


In [62]:
elec_day_test["rq"] = pd.to_datetime(elec_day_test["rq"])
elec_day_test = elec_day_test.groupby('id').apply(lambda x: x.sort_values('rq', ascending=True).fillna(method="ffill").fillna(method="bfill").fillna(0)).set_index(elec_day_test.index)
elec_day_test

,id,rq,kwh,kwh_rap,kwh_pap_r1,kwh_pap_r2,kwh_pap_r3,kwh_pap_r4
0,179404030,2020-01-22,51.77,0.0,0.0,33.47,0.0,18.31
1,179404030,2020-01-23,73.27,0.0,0.0,51.83,0.0,21.44
2,179404030,2020-01-24,74.90,0.0,0.0,52.20,0.0,22.70
3,179404030,2020-01-25,62.73,0.0,0.0,37.27,0.0,25.45
4,179404030,2020-01-26,64.27,0.0,0.0,41.72,0.0,22.56
...,...,...,...,...,...,...,...,...
1660582,2880712108,2021-10-05,44.85,0.0,0.0,22.14,0.0,22.71
1660583,2880712108,2021-10-06,28.89,0.0,0.0,14.94,0.0,13.95
1660584,2880712108,2021-10-07,51.10,0.0,0.0,27.27,0.0,23.84
1660585,2880712108,2021-10-08,17.96,0.0,0.0,7.19,0.0,10.76


<h4>手动特征工程</h4>

In [63]:
for cons_id in tqdm(cons_info_test.index):
    df = elec_day_test[elec_day_test["id"] == cons_id]
    if df.shape[0] == 0:
        continue
    df = df.drop(columns=["id"]).sort_values("rq").set_index("rq")
    df = df.fillna(method="ffill").fillna(method="bfill").fillna(0)
    
    cons_info_test.loc[cons_id, "kwh_mean"] = df["kwh"].mean()
    cons_info_test.loc[cons_id, "kwh_median"] = df["kwh"].median()
    cons_info_test.loc[cons_id, "kwh_std"] = df["kwh"].std()
    cons_info_test.loc[(cons_info_test.index == cons_id) & (cons_info_test["kwh_mean"] > 0), "kwh_std/mean"] = cons_info_test["kwh_std"] / cons_info_test["kwh_mean"]
    cons_info_test.loc[cons_id, "kwh_entropy"] = entropy(df["kwh"])

    cons_info_test.loc[cons_id, "kwh_pap_r23_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info_test.loc[cons_id, "kwh_pap_r24_distance"] = distance.euclidean(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info_test.loc[cons_id, "kwh_pap_r34_distance"] = distance.euclidean(df["kwh_pap_r3"], df["kwh_pap_r4"])

    cons_info_test.loc[cons_id, "kwh_pap_r23_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r3"])
    cons_info_test.loc[cons_id, "kwh_pap_r24_cosine"] = cosine(df["kwh_pap_r2"], df["kwh_pap_r4"])
    cons_info_test.loc[cons_id, "kwh_pap_r34_cosine"] = cosine(df["kwh_pap_r3"], df["kwh_pap_r4"])
    
    cons_info_test.loc[cons_id, "kwh_pap_r1_cosine"] = cosine(df["kwh_pap_r1"], df["kwh"])
    cons_info_test.loc[cons_id, "kwh_pap_r2_cosine"] = cosine(df["kwh_pap_r2"], df["kwh"])
    cons_info_test.loc[cons_id, "kwh_pap_r3_cosine"] = cosine(df["kwh_pap_r3"], df["kwh"])
    cons_info_test.loc[cons_id, "kwh_pap_r4_cosine"] = cosine(df["kwh_pap_r4"], df["kwh"])
    
    cons_info_test.loc[cons_id, "variation_day"] = variation(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    cons_info_test.loc[cons_id, "kwh_max"] = df["kwh"].max()
    cons_info_test.loc[cons_id, "kwh_min"] = df["kwh"].min()
    
    cons_info_test.loc[cons_id, "entropy_day"] = entropy_df(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    cons_info_test.loc[cons_id, "std_day"] = std_df(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    cons_info_test.loc[cons_id, "diff_day"] = diff_median(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    cons_info_test.loc[cons_id, "devide_day"] = devide_median(df[["kwh_pap_r2", "kwh_pap_r3", "kwh_pap_r4"]])
    
    
cons_info_test

100%|██████████████████████████████████████████████████████████████████████████████| 15379/15379 [09:38<00:00, 26.59it/s]


,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,pq_z_mean,pq_z_median,pq_z_std,pq_z_std/mean,...,kwh_pap_r2_cosine,kwh_pap_r3_cosine,kwh_pap_r4_cosine,variation_day,kwh_max,kwh_min,entropy_day,std_day,diff_day,devide_day
179406029,2.0,2.0,1.0,24,8431,8431,815.863636,737.0,213.639281,0.261857,...,0.015134,1.000000,0.040248,1.018344,44.74,8.14,1.051547,8.175570,30.470,4.576500e+08
179406030,12.0,12.0,0.0,24,8431,8431,352.409091,305.5,186.512519,0.529250,...,1.000000,0.000000,1.000000,1.732051,27.12,1.26,0.636514,5.150820,13.980,0.000000e+00
179406094,2.0,2.0,1.0,24,8310,8310,590.409091,564.0,102.925150,0.174329,...,0.000859,1.000000,0.080452,1.504231,26.22,13.38,0.807662,9.365858,33.140,5.034000e+08
179406097,4.0,4.0,1.0,36,8341,8341,676.727273,637.5,250.357874,0.369954,...,0.016689,1.000000,0.044281,1.044709,40.02,8.38,1.047268,7.110605,26.410,4.102500e+08
179406099,2.0,2.0,1.0,24,8431,8431,396.772727,399.0,183.313141,0.462010,...,0.009680,1.000000,0.035987,1.016197,39.62,0.10,1.064383,3.716477,11.880,1.726500e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,222.545455,197.0,90.766434,0.407856,...,0.402164,1.000000,0.064085,1.193530,39.04,0.00,0.245352,3.042272,0.000,0.000000e+00
2850017469,120.0,120.0,0.0,120,773,773,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.00,0.00,0.000000,0.000000,0.000,0.000000e+00
2850017472,720.0,720.0,0.0,120,773,773,23660.727273,24569.0,8178.414878,0.345654,...,0.000969,0.001243,0.004624,0.240959,1148.00,472.00,0.991645,60.598379,211.000,3.282523e+00
2851323065,12.0,12.0,0.0,120,780,780,210.227273,167.5,217.211973,1.033225,...,0.004505,1.000000,0.133719,1.218578,24.94,0.00,0.649498,2.426968,4.065,6.540000e+07


<h4>tsfresh特征工程</h4>

In [64]:
day_test_ts_features = extract_features(
    timeseries_container=elec_day_test,
    default_fc_parameters=EfficientFCParameters(),
    column_id="id",
    column_sort="rq",
    n_jobs=12
).fillna(0)
day_test_ts_features

Feature Extraction: 100%|████████████████████████████████████████████████████████████████| 60/60 [12:30<00:00, 12.51s/it]


variable,kwh__abs_energy,kwh__absolute_sum_of_changes,"kwh__agg_autocorrelation__f_agg_""mean""__maxlag_40","kwh__agg_autocorrelation__f_agg_""median""__maxlag_40","kwh__agg_autocorrelation__f_agg_""var""__maxlag_40","kwh__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""max""","kwh__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""mean""","kwh__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""min""","kwh__agg_linear_trend__attr_""intercept""__chunk_len_10__f_agg_""var""","kwh__agg_linear_trend__attr_""intercept""__chunk_len_50__f_agg_""max""",...,kwh_rap__symmetry_looking__r_0.9500000000000001,kwh_rap__time_reversal_asymmetry_statistic__lag_1,kwh_rap__time_reversal_asymmetry_statistic__lag_2,kwh_rap__time_reversal_asymmetry_statistic__lag_3,kwh_rap__value_count__value_-1,kwh_rap__value_count__value_0,kwh_rap__value_count__value_1,kwh_rap__variance,kwh_rap__variance_larger_than_standard_deviation,kwh_rap__variation_coefficient
id,,,,,,,,,,,,,,,,,,,,,
179404030,467715.9106,951.46,0.166839,0.144814,0.165347,33.636364,16.960443,7.649545,2.589082,83.950000,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
179404040,54430.6133,581.32,0.029237,-0.038378,0.065205,45.635000,27.750636,12.628636,97.618741,57.701667,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
179404046,785418.5622,1380.59,0.074002,0.049935,0.035073,98.269545,72.296636,55.751818,265.642458,144.540000,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
179404049,198460.8639,600.79,-0.056095,-0.090116,0.061829,47.873182,39.523261,28.844091,38.090298,67.045000,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
179404054,18543.3513,291.83,-0.009558,-0.020451,0.098747,11.246818,5.791739,1.042273,11.646500,26.403333,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2877297622,2116.8178,180.01,-0.004597,-0.028507,0.010148,14.602273,3.750080,0.251364,35.379902,27.863333,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
2878310631,2455.1838,44.55,0.504307,0.557225,0.116933,0.773636,-0.855898,-1.075000,0.808060,5.775000,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
2880078775,71143.5935,1178.47,-0.002405,-0.018812,0.028412,21.640455,6.195273,2.181818,22.014531,68.386667,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0


<h4>合并特征</h4>

In [65]:
cons_info_test = cons_info_test.merge(day_test_ts_features, left_index=True, right_index=True, how="left").fillna(0)
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,pq_z_mean,pq_z_median,pq_z_std,pq_z_std/mean,...,kwh_rap__symmetry_looking__r_0.9500000000000001,kwh_rap__time_reversal_asymmetry_statistic__lag_1,kwh_rap__time_reversal_asymmetry_statistic__lag_2,kwh_rap__time_reversal_asymmetry_statistic__lag_3,kwh_rap__value_count__value_-1,kwh_rap__value_count__value_0,kwh_rap__value_count__value_1,kwh_rap__variance,kwh_rap__variance_larger_than_standard_deviation,kwh_rap__variation_coefficient
179406029,2.0,2.0,1.0,24,8431,8431,815.863636,737.0,213.639281,0.261857,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
179406030,12.0,12.0,0.0,24,8431,8431,352.409091,305.5,186.512519,0.529250,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
179406094,2.0,2.0,1.0,24,8310,8310,590.409091,564.0,102.925150,0.174329,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
179406097,4.0,4.0,1.0,36,8341,8341,676.727273,637.5,250.357874,0.369954,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
179406099,2.0,2.0,1.0,24,8431,8431,396.772727,399.0,183.313141,0.462010,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,222.545455,197.0,90.766434,0.407856,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
2850017469,120.0,120.0,0.0,120,773,773,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
2850017472,720.0,720.0,0.0,120,773,773,23660.727273,24569.0,8178.414878,0.345654,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0
2851323065,12.0,12.0,0.0,120,780,780,210.227273,167.5,217.211973,1.033225,...,0.0,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0


In [66]:
cons_info_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15379 entries, 179406029 to 2852368013
Columns: 7621 entries, CONTRACT_CAP to kwh_rap__variation_coefficient
dtypes: float64(7616), int64(5)
memory usage: 894.9 MB


<h2>重新排列测试集</h2>

In [67]:
cons_info_test = cons_info_test.reindex(columns=cons_info.columns).drop(columns=["IS_FLAG"])
cons_info_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15379 entries, 179406029 to 2852368013
Columns: 7621 entries, CONTRACT_CAP to kwh_rap__variation_coefficient
dtypes: float64(7616), int64(5)
memory usage: 894.9 MB


In [68]:
x_test = ss.transform(cons_info_test.values)
x_test.shape

(15379, 7621)

In [69]:
x_test = feature_sel.transform(x_test)
x_test.shape

(15379, 100)

In [70]:
x_test

array([[-0.34570468, -2.04863254,  0.49088743, ...,  1.30950684,
         0.05112882,  0.05914937],
       [-0.18021871, -2.04863254,  0.23667391, ..., -1.83948407,
        -0.11220189, -0.12183483],
       [-0.34570468, -2.04863254,  0.57987573, ...,  0.95961896,
        -0.08341482, -0.09026502],
       ...,
       [11.53618784, -0.73630723,  1.53920571, ...,  0.60973108,
         2.77714728,  2.55316463],
       [-0.18021871, -0.73630723, -0.10980509, ..., -0.09004467,
        -0.1000166 , -0.09268698],
       [ 0.48999946, -0.73630723, -0.60433495, ...,  0.60973108,
         0.39223697,  0.37364255]])

<h2>预测数据</h2>

In [71]:
y_test = model.predict(x_test)
cons_info_test["label"] = y_test
cons_info_test

,CONTRACT_CAP,RUN_CAP,SHIFT_NO,CHK_CYCLE,live_days,check_days,pq_z_mean,pq_z_median,pq_z_std,pq_z_std/mean,...,kwh_rap__time_reversal_asymmetry_statistic__lag_1,kwh_rap__time_reversal_asymmetry_statistic__lag_2,kwh_rap__time_reversal_asymmetry_statistic__lag_3,kwh_rap__value_count__value_-1,kwh_rap__value_count__value_0,kwh_rap__value_count__value_1,kwh_rap__variance,kwh_rap__variance_larger_than_standard_deviation,kwh_rap__variation_coefficient,label
179406029,2.0,2.0,1.0,24,8431,8431,815.863636,737.0,213.639281,0.261857,...,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0,0
179406030,12.0,12.0,0.0,24,8431,8431,352.409091,305.5,186.512519,0.529250,...,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0,0
179406094,2.0,2.0,1.0,24,8310,8310,590.409091,564.0,102.925150,0.174329,...,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0,0
179406097,4.0,4.0,1.0,36,8341,8341,676.727273,637.5,250.357874,0.369954,...,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0,0
179406099,2.0,2.0,1.0,24,8431,8431,396.772727,399.0,183.313141,0.462010,...,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2849971818,12.0,12.0,0.0,120,783,783,222.545455,197.0,90.766434,0.407856,...,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0,0
2850017469,120.0,120.0,0.0,120,773,773,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0,0
2850017472,720.0,720.0,0.0,120,773,773,23660.727273,24569.0,8178.414878,0.345654,...,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0,0
2851323065,12.0,12.0,0.0,120,780,780,210.227273,167.5,217.211973,1.033225,...,0.0,0.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0,0


In [72]:
cons_info_test.index.name = "id"
cons_info_test[["label"]].to_csv("../data/result20220227-temp.csv", index=True)